## Logistic Regression

### Part a
Use a gradient descent method to optimize the logistic regression (LR) objective, with L2 regularization on the weight vector. For L2 regularization, the objective function (error of LR) should be of the form

$$ E_{LR}(w, w_0) = -\mathcal{L}(w, w_0) + \lambda \parallel w\parallel_2^2$$
where
$$ \parallel w\parallel = \sqrt{(w_1^2 + ... w_n^2)}$$
and
$$ \mathcal{L}(w, w_0) = -\sum_i{\log{(1+\exp(-y_i(w^Tx_i + w_0)))}} $$
Using your code to perform gradient descent, train your model parameters (w, w0) using the training data from data1 train.csv with $\lambda = 0$.

1. What happens to the weight vector as a function of the number of iteration of gradient descent?
2. What happens when λ = 1? Explain.

In [42]:
import numpy as np
a = np.array([[4.0, 1.0], [3.0, 2.0]])
b = 2*np.ones((2,1))

a = np.random.randn(3,1)

[[ 1.10490566]
 [ 0.31328433]
 [-0.21756748]]


In [69]:
import pdb
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
from mpl_toolkits.mplot3d import Axes3D

    
# parameters
name = '1'
print '======Training======'
# load data from csv files
train = np.loadtxt('hw2_resources/data/data'+name+'_train.csv')
X = train[:,0:2]
Y = train[:,2:3]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:,0], X[:, 1], Y[:, 0])
X = np.c_[np.ones(X.shape[0]), X]
print X.shape
print Y.shape
w = np.zeros((X.shape[1],1)) # Initialize w
print w.shape

# X is data matrix (each row is a data point)
# Y is desired output (1 or -1)
# scoreFn is a function of a data point
# values is a list of values to plot

def plotDecisionBoundary(X, Y, scoreFn, values, title = ""):
    # Plot the decision boundary. For that, we will asign a score to
    # each point in the mesh [x_min, m_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    h = max((x_max-x_min)/200., (y_max-y_min)/200.)
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                      np.arange(y_min, y_max, h))
    zz = np.array([scoreFn(x) for x in np.c_[xx.ravel(), yy.ravel()]])
    zz = zz.reshape(xx.shape)
    pl.figure()
    CS = pl.contour(xx, yy, zz, values, colors = 'green', linestyles = 'solid', linewidths = 2)
    pl.clabel(CS, fontsize=9, inline=1)
    # Plot the training points
    pl.scatter(X[:, 0], X[:, 1], c=(1.-Y), s=50, cmap = pl.cm.cool)
    pl.title(title)
    pl.axis('tight')


# Carry out training.
### TODO ###
def batch_gradient_descent(fobj, fgrad, init, lambd=0.0, alpha=1e-6, eps=1e-2):
    x = init
    fgradv = []
    xv = []
    fv = []
    curr_fx = fobj(x, lambd)
    for i in range(1,10000) :
        grad = fgrad(x, lambd)
        prev_fx = curr_fx
        
        # Parameter update
        x = x - np.multiply(alpha, grad)
        # Store all the values for viz
        fgradv.append(grad)
        xv.append(x)
        fv.append(curr_fx)

        #Update the current value
        curr_fx = fobj(x, lambd)
        delta_fx = np.fabs(curr_fx - prev_fx)
        if (delta_fx < eps):
            break

    return xv, fgradv, fv

def approx_gradient_method(func, x, delta):
    # This is the alternate method
    #grad = (func(x+dv) - func(x-dv))/(2.0*delta)
    dv = np.eye(x.shape[0], dtype=float)
    dv = np.multiply(dv, delta)
    grad = np.empty(x.shape[0])
    for i in range(x.shape[0]):
        print func(x+dv[i])
        grad[i] = (func(x+dv[i]) - func(x-dv[i]))/(2.0*delta)
    return grad

def logisticLossGrad(w, lambd=0.0):
    gradw_num = Y*X
    grad_denom = 1+np.exp(Y*(np.dot(X, w)))
    grad_div = np.divide(gradw_num, grad_denom)
    gradw = np.sum(grad_div, axis=0)
    gradw = np.reshape(gradw, (-1,1)) + 2.0*lambd*w
    return gradw

def logisticLoss(w, lambd=0.0):
    Y_hat = np.dot(X, w)
    loss = -np.sum(np.log(1+np.exp(-Y*Y_hat))) + lambd*np.dot(w.T, w)
    return loss

# import your LR training code
def eLR(w, b):
    return

# Define the predictLR(x) function, which uses trained parameters
# this is using the w and w0 obtained by minimizing loss
def predictLR(x) :
    y_pred = -1.0
    if (x.ndim == 1):
        x = np.reshape(x, (1,x.shape[0]))
        x = np.c_[np.ones(1), x]
        y_pred = np.dot(x, w)
    else:
        y_pred = np.dot(x, w[-1])
    if y_pred[0][0] >= 0.0:
        return 1.0
    else:
        return -1.0

approx_grad = approx_gradient_method(logisticLoss, w, 1e-3)
compute_grad = logisticLossGrad(w)

print approx_grad
print compute_grad

wv1,_,_ = batch_gradient_descent(logisticLoss, logisticLossGrad, w, 1.0)
wv,_,_ = batch_gradient_descent(logisticLoss, logisticLossGrad, w)

w = wv[-1]
print w

# plot training results
plotDecisionBoundary(X, Y, predictLR, [0.5], title = 'LR Train')

print '======Validation======'
# load data from csv files
validate = np.loadtxt('hw2_resources/data/data'+name+'_validate.csv')
X = validate[:,0:2]
Y = validate[:,2:3]

# plot validation results
plotDecisionBoundary(X, Y, predictLR, [0.5], title = 'LR Validate')
pl.show()


======Training======
(400, 3)
(400, 1)
(3, 1)
[[-831.3878842 -831.3878842 -831.3878842]
 [-831.3878842 -831.3878842 -831.3878842]
 [-831.3878842 -831.3878842 -831.3878842]]


ValueError: setting an array element with a sequence.